# Sympy

In [ ]:
import sympy as sp
import numpy as np

In [ ]:
alg_inputs = []

# PROGRAMMING: Do this with Drake instead?
# PROGRAMMING: Can I get some of these lists from sympy?
# Physical and geometric quantities
m_L = sp.symbols(r"m_L")
alg_inputs.append(m_L)
m_M = sp.symbols(r"m_M")
alg_inputs.append(m_M)
w_L = sp.symbols(r"w_L")
alg_inputs.append(w_L)
I_L = sp.symbols(r"I_L")
alg_inputs.append(I_L)
h_L = sp.symbols(r"h_L")
alg_inputs.append(h_L)
r = sp.symbols(r"r")
alg_inputs.append(r)
I_M = sp.symbols(r"I_M")
alg_inputs.append(I_M)

In [ ]:
# Friction coefficients
mu = sp.symbols(r"\mu")
alg_inputs.append(mu)
mu_S = sp.symbols(r"\mu_{S}")
alg_inputs.append(mu_S)

# System gains
b_J = sp.symbols(r"b_J")
alg_inputs.append(b_J)
k_J = sp.symbols(r"k_J")
alg_inputs.append(k_J)

In [ ]:
# Positions
p_CN = sp.symbols(r"p_{CN}")
alg_inputs.append(p_CN)
p_CT = sp.symbols(r"p_{CT}")
alg_inputs.append(p_CT)
p_MN = sp.symbols(r"p_{MN}")
alg_inputs.append(p_MN)
p_LN = sp.symbols(r"p_{LN}")
alg_inputs.append(p_LN)
p_LT = sp.symbols(r"p_{LT}")
alg_inputs.append(p_LT)
theta_L = sp.symbols(r"\theta_L")
alg_inputs.append(theta_L)
d_T = sp.symbols(r"d_T")
alg_inputs.append(d_T)
d_N = sp.symbols(r"d_N")
alg_inputs.append(d_N)

# Velocities
v_MN = sp.symbols(r"v_{MN}")
alg_inputs.append(v_MN)
v_MT = sp.symbols(r"v_{MT}")
alg_inputs.append(v_MT)
v_LN = sp.symbols(r"v_{LN}")
alg_inputs.append(v_LN)
v_LT = sp.symbols(r"v_{LT}")
alg_inputs.append(v_LT)
d_theta_L = sp.symbols(r"\dot\theta_L")
alg_inputs.append(d_theta_L)
d_theta_M = sp.symbols(r"\dot\theta_M")
alg_inputs.append(d_theta_M)
d_d_T = sp.symbols(r"\dot{d}_T")
alg_inputs.append(d_d_T)
d_d_N = sp.symbols(r"\dot{d}_N")
alg_inputs.append(d_d_N)

In [ ]:
# Input forces
F_GT = sp.symbols(r"F_{GT}")
alg_inputs.append(F_GT)
F_GN = sp.symbols(r"F_{GN}")
alg_inputs.append(F_GN)
F_OT, F_ON, tau_O = sp.symbols(r"F_{OT}, F_{ON} \tau_O")
alg_inputs.append(F_OT)
alg_inputs.append(F_ON)
alg_inputs.append(tau_O)

# Control inputs
dd_theta_Ld = sp.symbols(r"\ddot\theta_{Ld}")
alg_inputs.append(dd_theta_Ld)
dd_d_Nd = sp.symbols(r"\ddot{d}_{Nd}")
alg_inputs.append(dd_d_Nd)
dd_d_Td = sp.symbols(r"\ddot{d}_{Td}")
alg_inputs.append(dd_d_Td)
dd_theta_Md = sp.symbols(r"\ddot\theta_{Md}")
alg_inputs.append(dd_theta_Md)

outputs = [
    a_LT, a_LN, a_MT, a_MN, dd_theta_L, dd_theta_M, F_NM, F_FL, F_FM, dd_d_N, dd_d_T, F_NL, F_CN, F_CT, tau_M
] = sp.symbols(
    r"a_{LT} a_{LN} a_{MT} a_{MN} \ddot\theta_L \ddot\theta_M  F_{NM} F_{FL} F_{FM} \ddot{d}_N \ddot{d}_T F_{NL} F_{CN}, F_{CT} \tau_M"
)

outputs = [
    a_LT, a_LN, a_MT, a_MN, dd_theta_L, dd_theta_M, F_NM, F_FL, F_FM, dd_d_N, dd_d_T, F_NL, F_CN, F_CT, tau_M
] = sp.symbols(
    r"a_{LT} a_{LN} a_{MT} a_{MN} \ddot\theta_L \ddot\theta_M  F_{NM} F_{FL} F_{FM} \ddot{d}_N \ddot{d}_T F_{NL} F_{CN}, F_{CT} \tau_M"
)

outputs = list(outputs)

In [ ]:
t = sp.symbols("t")
theta_L_func = sp.Function(r'\theta_L')(t)
N_hat = sp.Function(r'\hat N')(theta_L_func)
T_hat = sp.Function(r'\hat T')(theta_L_func)

d_T_func = sp.Function(r"d_T")(t)
d_N_func = sp.Function(r"d_N")(t)
d_g = d_T_func*T_hat + d_N_func*N_hat

d_vel_g = sp.diff(d_g, t)

d_vel_g = d_vel_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_g = d_vel_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_g = sp.diff(d_vel_g, t)
d_acc_g = d_acc_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_g = d_acc_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_g = d_acc_g
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_g = d_acc_cos_g.subs(d_T_func, d_T)
d_acc_cos_g = d_acc_cos_g.subs(d_N_func, d_N)

In [ ]:
dd_d_g_T = d_acc_cos_g.subs(N_hat, 0).subs(T_hat, 1)
dd_d_g_T

In [ ]:
dd_d_g_N = d_acc_cos_g.subs(T_hat, 0).subs(N_hat, 1)
dd_d_g_N

In [ ]:
p_M_func = sp.Function(r"p_M")(t)
p_L_func = sp.Function(r"p_L")(t)
v_M = sp.symbols(r"v_M")
v_L = sp.symbols(r"v_L")
d_s = (p_M_func + r*N_hat) - (p_L_func + (w_L/2)*T_hat - (h_L/2)*N_hat)

d_vel_s = sp.diff(d_s, t)
d_vel_s = d_vel_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_s = d_vel_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_s = sp.diff(d_vel_s, t)
d_acc_s = d_acc_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_s = d_acc_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_s = d_acc_s
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_s = d_acc_cos_s.subs(d_T_func, d_T)
d_acc_cos_s = d_acc_cos_s.subs(d_N_func, d_N)

In [ ]:
dd_d_s_T = d_acc_cos_s.subs(N_hat, 0).subs(T_hat, 1)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_L_func, t, t), a_LT)
dd_d_s_T

In [ ]:
dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)

In [ ]:
dd_d_s_N = d_acc_cos_s.subs(T_hat, 0).subs(N_hat, 1)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_M_func, t, t), a_MN)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_L_func, t, t), a_LN)
dd_d_s_N

In [ ]:
nat_eqs = [
    # Link tangential force balance
    [m_L*a_LT, F_FL+F_GT+F_OT],
    # Link normal force balance
    [m_L*a_LN, F_NL + F_GN + F_ON, ],
    # Manipulator tangential force balance
    [m_M*a_MT, F_FM + F_CT, ],
    # Manipulator normal force balance
    [m_M*a_MN, F_NM+F_CN, ],
    # Link moment balance
    [I_L*dd_theta_L, (-w_L/2)*F_ON - (p_CN-p_LN) * \
     F_FL + (p_CT-p_LT)*F_NL + tau_O, ],
    # Manipulator moment balance
    [I_M*dd_theta_M, tau_M-F_FM*(p_CN-p_MN), ],
#     # 3rd law normal forces
#     [F_NL, -F_NM],
    # Friction relationship L
    [F_FL, mu*mu_S*F_NL],
#     # Friction relationship M
#     [F_FM, -F_FL],
    # d_T derivative is derivative
    [dd_d_s_T, dd_d_g_T],
    # d_N derivative is derivative
    [dd_d_s_N, dd_d_g_N],
]

art_eqs = [
    [dd_d_N, dd_d_Nd],
    [dd_d_T, dd_d_Td],
    [dd_theta_L, dd_theta_Ld],
    [dd_theta_M, dd_theta_Md],
]

env_eqs = nat_eqs# + art_eqs

In [ ]:
A = []
b = []
for lhs, rhs in env_eqs:
    A_row = []
    b_term = rhs - lhs
    for output_term in outputs:
        try:
            coeff_L = lhs.coeff(output_term)
        except AttributeError:
            coeff_L = 0
        try:
            coeff_R = rhs.coeff(output_term)
        except AttributeError:
            coeff_R = 0
        coeff = coeff_L - coeff_R
        A_row.append(coeff)
        b_term += coeff * output_term
    A.append(A_row)
    b.append(b_term)
A = sp.SparseMatrix(A)
A.simplify()
b = sp.Matrix([b]).T
b.simplify()
x = sp.Matrix([outputs]).T
x.simplify()

In [ ]:
sp.SparseMatrix

In [ ]:
A.shape

In [ ]:
# A_p = A.pinv()

In [ ]:
A_intermed = A@A.T

In [ ]:
AA_i = A_intermed.inv()

In [ ]:
# A_aug = A.row_join(b)
# results = A_aug.rref()[0]
# A_prime = results[:, :-1]
# b_prime = results[:, -1]

# assert A_prime == sp.eye(A_prime.shape[0])

# F_CN_idx = list(x).index(F_CN)
# F_CN_exp = b_prime[F_CN_idx]
# F_CT_idx = list(x).index(F_CT)
# F_CT_exp = b_prime[F_CT_idx]
# tau_M_idx = list(x).index(tau_M)
# tau_M_exp = b_prime[tau_M_idx]

# get_F_CN = lambdify([alg_inputs], F_CN_exp)
# get_F_CT = lambdify([alg_inputs], F_CT_exp)
# get_tau_M = lambdify([alg_inputs], tau_M_exp)

# tau_M_idx = list(x).index(tau_M)
# tau_M_exp = b_prime[tau_M_idx]

# Sympy 2

In [ ]:
import sympy as sp
import numpy as np

In [ ]:
alg_inputs = []

# PROGRAMMING: Do this with Drake instead?
# PROGRAMMING: Can I get some of these lists from sympy?
# Physical and geometric quantities
m_L = sp.symbols(r"m_L")
alg_inputs.append(m_L)
m_M = sp.symbols(r"m_M")
alg_inputs.append(m_M)
w_L = sp.symbols(r"w_L")
alg_inputs.append(w_L)
I_L = sp.symbols(r"I_L")
alg_inputs.append(I_L)
h_L = sp.symbols(r"h_L")
alg_inputs.append(h_L)
r = sp.symbols(r"r")
alg_inputs.append(r)
I_M = sp.symbols(r"I_M")
alg_inputs.append(I_M)

In [ ]:
# Friction coefficients
mu = sp.symbols(r"\mu")
alg_inputs.append(mu)
mu_S = sp.symbols(r"\mu_{S}")
alg_inputs.append(mu_S)

# System gains
b_J = sp.symbols(r"b_J")
alg_inputs.append(b_J)
k_J = sp.symbols(r"k_J")
alg_inputs.append(k_J)

In [ ]:
# Positions
p_CN = sp.symbols(r"p_{CN}")
alg_inputs.append(p_CN)
p_CT = sp.symbols(r"p_{CT}")
alg_inputs.append(p_CT)
p_MN = sp.symbols(r"p_{MN}")
alg_inputs.append(p_MN)
p_LN = sp.symbols(r"p_{LN}")
alg_inputs.append(p_LN)
p_LT = sp.symbols(r"p_{LT}")
alg_inputs.append(p_LT)
theta_L = sp.symbols(r"\theta_L")
alg_inputs.append(theta_L)
d_T = sp.symbols(r"d_T")
alg_inputs.append(d_T)
d_N = sp.symbols(r"d_N")
alg_inputs.append(d_N)

# Velocities
v_MN = sp.symbols(r"v_{MN}")
alg_inputs.append(v_MN)
v_MT = sp.symbols(r"v_{MT}")
alg_inputs.append(v_MT)
v_LN = sp.symbols(r"v_{LN}")
alg_inputs.append(v_LN)
v_LT = sp.symbols(r"v_{LT}")
alg_inputs.append(v_LT)
d_theta_L = sp.symbols(r"\dot\theta_L")
alg_inputs.append(d_theta_L)
d_theta_M = sp.symbols(r"\dot\theta_M")
alg_inputs.append(d_theta_M)
d_d_T = sp.symbols(r"\dot{d}_T")
alg_inputs.append(d_d_T)
d_d_N = sp.symbols(r"\dot{d}_N")
alg_inputs.append(d_d_N)

In [ ]:
# Input forces
F_GT = sp.symbols(r"F_{GT}")
alg_inputs.append(F_GT)
F_GN = sp.symbols(r"F_{GN}")
alg_inputs.append(F_GN)
F_OT, F_ON, tau_O = sp.symbols(r"F_{OT}, F_{ON} \tau_O")
alg_inputs.append(F_OT)
alg_inputs.append(F_ON)
alg_inputs.append(tau_O)

# Control inputs
dd_theta_Ld = sp.symbols(r"\ddot\theta_{Ld}")
alg_inputs.append(dd_theta_Ld)
dd_d_Nd = sp.symbols(r"\ddot{d}_{Nd}")
alg_inputs.append(dd_d_Nd)
dd_d_Td = sp.symbols(r"\ddot{d}_{Td}")
alg_inputs.append(dd_d_Td)
dd_theta_Md = sp.symbols(r"\ddot\theta_{Md}")
alg_inputs.append(dd_theta_Md)

outputs = [
    a_LT, a_LN, a_MT, a_MN, dd_theta_L, dd_theta_M, F_FL, dd_d_N, dd_d_T, F_NL
] = sp.symbols(
    r"a_{LT} a_{LN} a_{MT} a_{MN} \ddot\theta_L \ddot\theta_M  F_{FL} \ddot{d}_N \ddot{d}_T F_{NL}"
)

[
    F_CN, F_CT
] = sp.symbols(
    r"F_{CN} F_{CT}"
)

outputs = list(outputs)

In [ ]:
t = sp.symbols("t")
theta_L_func = sp.Function(r'\theta_L')(t)
N_hat = sp.Function(r'\hat N')(theta_L_func)
T_hat = sp.Function(r'\hat T')(theta_L_func)

d_T_func = sp.Function(r"d_T")(t)
d_N_func = sp.Function(r"d_N")(t)
d_g = d_T_func*T_hat + d_N_func*N_hat

d_vel_g = sp.diff(d_g, t)

d_vel_g = d_vel_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_g = d_vel_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_g = sp.diff(d_vel_g, t)
d_acc_g = d_acc_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_g = d_acc_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_g = d_acc_g
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_g = d_acc_cos_g.subs(d_T_func, d_T)
d_acc_cos_g = d_acc_cos_g.subs(d_N_func, d_N)

In [ ]:
dd_d_g_T = d_acc_cos_g.subs(N_hat, 0).subs(T_hat, 1)
dd_d_g_T

In [ ]:
dd_d_g_N = d_acc_cos_g.subs(T_hat, 0).subs(N_hat, 1)
dd_d_g_N

In [ ]:
p_M_func = sp.Function(r"p_M")(t)
p_L_func = sp.Function(r"p_L")(t)
v_M = sp.symbols(r"v_M")
v_L = sp.symbols(r"v_L")
d_s = (p_M_func + r*N_hat) - (p_L_func + (w_L/2)*T_hat - (h_L/2)*N_hat)

d_vel_s = sp.diff(d_s, t)
d_vel_s = d_vel_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_s = d_vel_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_s = sp.diff(d_vel_s, t)
d_acc_s = d_acc_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_s = d_acc_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_s = d_acc_s
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_s = d_acc_cos_s.subs(d_T_func, d_T)
d_acc_cos_s = d_acc_cos_s.subs(d_N_func, d_N)

In [ ]:
dd_d_s_T = d_acc_cos_s.subs(N_hat, 0).subs(T_hat, 1)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_L_func, t, t), a_LT)
dd_d_s_T

In [ ]:
dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)

In [ ]:
dd_d_s_N = d_acc_cos_s.subs(T_hat, 0).subs(N_hat, 1)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_M_func, t, t), a_MN)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_L_func, t, t), a_LN)
dd_d_s_N

In [ ]:
nat_eqs = [
    # Link tangential force balance
    [m_L*a_LT, F_FL+F_GT+F_OT],
    # Link normal force balance
    [m_L*a_LN, F_NL + F_GN + F_ON, ],
    # Manipulator tangential force balance
    [m_M*a_MT, (-F_FL) + F_CT, ],
    # Manipulator normal force balance
    [m_M*a_MN, (-F_NL)+F_CN, ],
    # Link moment balance
    [I_L*dd_theta_L, (-w_L/2)*F_ON - (p_CN-p_LN) * \
     F_FL + (p_CT-p_LT)*F_NL + tau_O, ],
    # Friction relationship L
    [F_FL, mu*mu_S*F_NL],
    # d_T derivative is derivative
    [dd_d_s_T, dd_d_g_T],
    # d_N derivative is derivative
    [dd_d_s_N, dd_d_g_N],
]
env_eqs = nat_eqs# + art_eqs

In [ ]:
A = []
b = []
for lhs, rhs in env_eqs:
    A_row = []
    b_term = rhs - lhs
    for output_term in outputs:
        try:
            coeff_L = lhs.coeff(output_term)
        except AttributeError:
            coeff_L = 0
        try:
            coeff_R = rhs.coeff(output_term)
        except AttributeError:
            coeff_R = 0
        coeff = coeff_L - coeff_R
        A_row.append(coeff)
        b_term += coeff * output_term
    A.append(A_row)
    b.append(b_term)
A = sp.Matrix(A)
A.simplify()
b = sp.Matrix([b]).T
b.simplify()
x = sp.Matrix([outputs]).T
x.simplify()

In [ ]:
x

In [ ]:
sp.SparseMatrix

In [ ]:
type(A)

In [ ]:
A

In [ ]:
A_intermed = A@A.T

In [ ]:
A_intermed

## Manually computing

In [ ]:
stacked = A_intermed.row_join(sp.eye(A.shape[0]))

In [ ]:
manual_inv = sp.eye(A.shape[0])

In [ ]:
def swap_rows(M, i, j):
    ret = 1*M
    ret[i,:] = M[j,:]
    ret[j,:] = M[i,:]
    return ret

In [ ]:
def mult_row(M, i, alpha):
    mult = sp.eye(M.shape[0])
    mult[i,i] = alpha
    return mult@M

In [ ]:
def add_row(M, i, j):
    ret = 1*M
    ret[j,:] += M[i,:]
    return ret

In [ ]:
def add_row_mult(M, i, j, alpha):
    ret = 1*M
    ret[j,:] += alpha*M[i,:]
    return ret

In [ ]:
# TODO: automate this
out = 1*stacked.subs(m_L, 3).subs(m_M, 5).subs(I_L, 2).subs(I_M, 7).subs(w_L, 11).subs(h_L, 11)
out.simplify()
out = mult_row(out, 0, 1/(m_L**2+1))
# out = mult_row(out, 1, 1/(m_L**2+1))
# out = add_row(out, 0, 2)
# out = mult_row(out, 2, 1/(m_M**2+1-1/(m_L**2+1)))
# out = add_row(out, 1, 3)
# out = mult_row(out, 3, 1/(m_M**2+1-1/(m_L**2+1)))
# # out.simplify()
# out = add_row_mult(out, 2, 0, 1/(m_L**2+1))
# out = add_row_mult(out, 3, 1, 1/(m_L**2+1))
# out = add_row_mult(out, 0, 4, -stacked[4,0])
# out = add_row_mult(out, 1, 4, -stacked[4,1])
# out = add_row_mult(out, 2, 4, -stacked[4,2])
# # out.simplify()
# out = add_row_mult(out, 3, 4, -stacked[4,3])
# out = mult_row(out, 4, 1/out[4,4])
# out = add_row_mult(out, 4, 3, -out[3,4])
# out = add_row_mult(out, 4, 2, -out[2,4])
# out = add_row_mult(out, 4, 1, -out[1,4])
# out = add_row_mult(out, 4, 0, -out[0,4])
# # out.simplify()
# out = add_row(out, 0, 5)
# out = add_row_mult(out, 1, 5, -mu*mu_S)
# out = add_row(out, 2, 5)
# out = add_row_mult(out, 3, 5, mu*mu_S)
# out = add_row_mult(out, 4, 5, -(mu*mu_S*(p_CT-p_LT)+p_CN-p_LN))
# out.simplify()
out

In [ ]:
# TODO: automate this
sub_stacked = stacked.subs(m_L, 3).subs(m_M, 5).subs(I_L, 2).subs(I_M, 7).subs(w_L, 11).subs(h_L, 11)
out = 1*sub_stacked
out = mult_row(out, 0, 1/sub_stacked[0,0])
out = mult_row(out, 1, 1/sub_stacked[1,1])
out = add_row(out, 0, 2)
out = mult_row(out, 2, 1/out[2,2])
out = add_row_mult(out, 2, 0, -out[0,2])
out = add_row(out, 1, 3)
out = mult_row(out, 3, 1/out[3,3])
out = add_row_mult(out, 3, 1, -out[1,3])
out = add_row_mult(out, 0, 4, -stacked[4,0])
out = add_row_mult(out, 1, 4, -stacked[4,1])
out = add_row_mult(out, 2, 4, -stacked[4,2])
out = add_row_mult(out, 3, 4, -stacked[4,3])
out = mult_row(out, 4, 1/out[4,4])
out = add_row_mult(out, 4, 3, -out[3,4])
out = add_row_mult(out, 4, 2, -out[2,4])
out = add_row_mult(out, 4, 1, -out[1,4])
out = add_row_mult(out, 4, 0, -out[0,4])
# out.simplify()
out = add_row(out, 0, 5)
out = add_row_mult(out, 1, 5, -out[5,1])
out = add_row_mult(out, 2, 5, -1)
out = add_row_mult(out, 3, 5, -out[5,3])
out = add_row_mult(out, 4, 5, -out[5,4])
out = add_row_mult(out, 0, 6, -out[6,0])
out = add_row_mult(out, 2, 6, -out[6,2])
out = add_row_mult(out, 4, 6, -out[6,4])
out = mult_row(out, 5, 1/out[5,5])
out = add_row_mult(out, 5, 4, -out[4,5])
out = add_row_mult(out, 5, 3, -out[3,5])
out = add_row_mult(out, 5, 2, -out[2,5])
out = add_row_mult(out, 5, 1, -out[1,5])
out = add_row_mult(out, 5, 0, -out[0,5])
out = add_row_mult(out, 5, 6, -out[6,5])
out = add_row_mult(out, 5, 7, -out[7,5])
out = add_row_mult(out, 1, 7, -out[7,1])
out = add_row_mult(out, 3, 7, -out[7,3])
out = add_row_mult(out, 4, 7, -out[7,4])
out = mult_row(out, 6, 1/out[6,6])
out = add_row_mult(out, 6, 5, -out[5,6])
out = add_row_mult(out, 6, 4, -out[4,6])
out = add_row_mult(out, 6, 3, -out[3,6])
out = add_row_mult(out, 6, 2, -out[2,6])
out = add_row_mult(out, 6, 1, -out[1,6])
out = add_row_mult(out, 6, 0, -out[0,6])
out = add_row_mult(out, 6, 7, -out[7,6])
out = mult_row(out, 7, 1/out[7,7])
out = add_row_mult(out, 7, 6, -out[6,7])
out = add_row_mult(out, 7, 5, -out[5,7])
out = add_row_mult(out, 7, 4, -out[4,7])
out = add_row_mult(out, 7, 3, -out[3,7])
out = add_row_mult(out, 7, 2, -out[2,7])
out = add_row_mult(out, 7, 1, -out[1,7])
out = add_row_mult(out, 7, 0, -out[0,7])

In [ ]:
out[0,0].coeff(mu)

In [ ]:
AA_i = out[:,8:]

In [ ]:
A_intermed.shape
stacked.sh

In [ ]:
A_mu = sp.zeros(A_intermed.shape[0], A_intermed.shape[1])
for i in range(A_intermed.shape[0]):
    for j in range(A_intermed.shape[1]):
        A_mu[i,j] = mu*(AA_i[i,j].coeff(mu))

In [ ]:
A_mu

In [ ]:
sub_A_intermed = A_intermed.subs(m_L, 3).subs(m_M, 5).subs(I_L, 2).subs(I_M, 7).subs(w_L, 11).subs(h_L, 11)

In [ ]:
full_sub_AA_i = AA_i.subs(mu_S, 13).subs(p_CN, 17).subs(p_CT, 19).subs(p_LN, 23).subs(p_LT, 29).subs(r, 31).subs(d_T, 37).subs(d_N, 41)
full_sub_AA_i.free_symbols

In [ ]:
full_sub_AA_i

In [ ]:
AA_i@(sub_A_intermed)

In [ ]:
out.subs(m_L, 3).subs(m_M, 5).subs(I_L, 2).subs(I_M, 7)

In [ ]:
A_intermed_sub = A_intermed.subs(m_L, 3).subs(m_M, 5).subs(I_L, 2).subs(I_M, 7).subs(w_L, 11).subs(h_L, 11)
AA_i = A_intermed_sub.inv()

In [ ]:
stacked.shape

In [ ]:

manual_inv@stacked

In [ ]:
AA_i = A_intermed.inv()

In [ ]:
# A_aug = A.row_join(b)
# results = A_aug.rref()[0]
# A_prime = results[:, :-1]
# b_prime = results[:, -1]

# assert A_prime == sp.eye(A_prime.shape[0])

# F_CN_idx = list(x).index(F_CN)
# F_CN_exp = b_prime[F_CN_idx]
# F_CT_idx = list(x).index(F_CT)
# F_CT_exp = b_prime[F_CT_idx]
# tau_M_idx = list(x).index(tau_M)
# tau_M_exp = b_prime[tau_M_idx]

# get_F_CN = lambdify([alg_inputs], F_CN_exp)
# get_F_CT = lambdify([alg_inputs], F_CT_exp)
# get_tau_M = lambdify([alg_inputs], tau_M_exp)

# tau_M_idx = list(x).index(tau_M)
# tau_M_exp = b_prime[tau_M_idx]

In [ ]:
A

# Drake

In [ ]:
from pydrake.all import MathematicalProgram, SnoptSolver
import pydrake.symbolic as sym

In [ ]:
prog = MathematicalProgram()

In [ ]:
# Physical and geometric quantities
m_L = sym.ExpressionConstant("m_L")
prog.AddIndeterminates(np.array([m_L]))
m_M = sym.Variable("m_M")
prog.AddIndeterminates(np.array([m_M]))
w_L = sym.Variable("w_L")
prog.AddIndeterminates(np.array([w_L]))
I_L = sym.Variable("I_L")
prog.AddIndeterminates(np.array([I_L]))
h_L = sym.Variable("h_L")
prog.AddIndeterminates(np.array([h_L]))
r = sym.Variable("r")
prog.AddIndeterminates(np.array([r]))
I_M = sym.Variable("I_M")
prog.AddIndeterminates(np.array([I_M]))

In [ ]:
# Friction coefficients
mu = sym.Variable("mu")
prog.AddIndeterminates(np.array([mu]))
mu_S = sym.Variable("mu_S")
prog.AddIndeterminates(np.array([mu_S]))

# System gains
b_J = sym.Variable("b_J")
prog.AddIndeterminates(np.array([b_J]))
k_J = sym.Variable("k_J")
prog.AddIndeterminates(np.array([k_J]))

In [ ]:
# Positions
p_CN = sym.Variable("p_CN")
prog.AddIndeterminates(np.array([p_CN]))
p_CT = sym.Variable("p_CT")
prog.AddIndeterminates(np.array([p_CT]))
p_MN = sym.Variable("p_MN")
prog.AddIndeterminates(np.array([p_MN]))
p_LN = sym.Variable("p_LN")
prog.AddIndeterminates(np.array([p_LN]))
p_LT = sym.Variable("p_LT")
prog.AddIndeterminates(np.array([p_LT]))
theta_L = sym.Variable("theta_L")
prog.AddIndeterminates(np.array([theta_L]))
d_T = sym.Variable("d_T")
prog.AddIndeterminates(np.array([d_T]))
d_N = sym.Variable("d_N")
prog.AddIndeterminates(np.array([d_N]))

# Velocities
v_MN = sym.Variable("v_MN")
prog.AddIndeterminates(np.array([v_MN]))
v_MT = sym.Variable("v_MT")
prog.AddIndeterminates(np.array([v_MT]))
v_LN = sym.Variable("v_LN")
prog.AddIndeterminates(np.array([v_LN]))
v_LT = sym.Variable("v_LT")
prog.AddIndeterminates(np.array([v_LT]))
d_theta_L = sym.Variable("d_theta_L")
prog.AddIndeterminates(np.array([d_theta_L]))
d_theta_M = sym.Variable("d_theta_M")
prog.AddIndeterminates(np.array([d_theta_M]))
d_d_T = sym.Variable("d_d_T")
prog.AddIndeterminates(np.array([d_d_T]))
d_d_N = sym.Variable("d_d_N")
prog.AddIndeterminates(np.array([d_d_N]))

In [ ]:
# Input forces
F_GT = sym.Variable("F_GT")
prog.AddDecisionVariables(np.array([F_GT]))
F_GN = sym.Variable("F_GN")
prog.AddDecisionVariables(np.array([F_GN]))
F_OT = sym.Variable("F_GN")
prog.AddDecisionVariables(np.array([F_OT]))
F_ON = sym.Variable("F_GN")
prog.AddDecisionVariables(np.array([F_ON]))
tau_O = sym.Variable("F_OT, F_ON, tau_O")
prog.AddDecisionVariables(np.array([tau_O]))

In [ ]:
# Outputs
a_LT = sym.Variable("a_LT")
prog.AddDecisionVariables(np.array([a_LT]))
a_LN = sym.Variable("a_LN")
prog.AddDecisionVariables(np.array([a_LN]))
a_MT = sym.Variable("a_MT")
prog.AddDecisionVariables(np.array([a_MT]))
a_MN = sym.Variable("a_MN")
prog.AddDecisionVariables(np.array([a_MN]))
dd_theta_L = sym.Variable("dd_theta_L")
prog.AddDecisionVariables(np.array([dd_theta_L]))
dd_theta_M = sym.Variable("dd_theta_M")
prog.AddDecisionVariables(np.array([dd_theta_M]))
F_NM = sym.Variable("F_NM")
prog.AddDecisionVariables(np.array([F_NM]))
F_FL = sym.Variable("F_FL")
prog.AddDecisionVariables(np.array([F_FL]))
F_FM = sym.Variable("F_FM")
prog.AddDecisionVariables(np.array([F_FM]))
dd_d_N = sym.Variable("dd_d_N")
prog.AddDecisionVariables(np.array([dd_d_N]))
dd_d_T = sym.Variable("dd_d_T")
prog.AddDecisionVariables(np.array([dd_d_T]))
F_NL = sym.Variable("F_NL")
prog.AddDecisionVariables(np.array([F_NL]))
F_CN = sym.Variable("F_CN")
prog.AddDecisionVariables(np.array([F_CN]))
F_CT = sym.Variable("F_CT")
prog.AddDecisionVariables(np.array([F_CT]))
tau_M = sym.Variable("tau_M")
prog.AddDecisionVariables(np.array([tau_M]))

In [ ]:
dd_d_g_T = -dd_theta_L*d_N + dd_d_T - d_theta_L**2*d_T - 2*d_theta_L*d_d_N
dd_d_g_N = dd_theta_L*d_T + dd_d_N - d_theta_L**2*d_N + 2*d_theta_L*d_d_T

In [ ]:
dd_d_s_T = -dd_theta_L*h_L/2 - dd_theta_L*r + d_theta_L**2*w_L/2 - a_LT + a_MT
dd_d_s_N = -dd_theta_L*w_L/2 - d_theta_L**2*h_L/2 - d_theta_L**2*r - a_LN + a_MN

In [ ]:
prog.AddLinearConstraint(a_LT == F_FL+F_GT+F_OT)
prog.AddLinearConstraint(a_LN == F_NL + F_GN + F_ON)
# prog.AddLinearConstraint(m_M*a_MT == F_FM + F_CT)
# prog.AddLinearConstraint(m_M*a_MN == F_NM+F_CN)
# prog.AddLinearConstraint(I_L*dd_theta_L == (-w_L/2)*F_ON - (p_CN-p_LN) * F_FL + (p_CT-p_LT)*F_NL + tau_O)
# prog.AddLinearConstraint(I_M*dd_theta_M == tau_M-F_FM*(p_CN-p_MN))
# prog.AddLinearConstraint(F_NL == -F_NM)
# prog.AddLinearConstraint(F_FL == mu*mu_S*F_NL)
# prog.AddLinearConstraint(F_FM == -F_FL)
# prog.AddLinearConstraint(dd_d_s_T == dd_d_g_T)
# prog.AddLinearConstraint(dd_d_s_N == dd_d_g_N)

In [ ]:
solver = SnoptSolver()

In [ ]:
res = solver.Solve(prog)

In [ ]:
res.GetSolution().size

In [ ]:
prog.LinearConstraint(m_L*a_LT == F_FL+F_GT+F_OT)
# prog.AddLinearConstraint(m_L*a_LN == F_NL + F_GN + F_ON)
# prog.AddLinearConstraint(m_M*a_MT == F_FM + F_CT)
# prog.AddLinearConstraint(m_M*a_MN == F_NM+F_CN)
# prog.AddLinearConstraint(I_L*dd_theta_L == (-w_L/2)*F_ON - (p_CN-p_LN) * F_FL + (p_CT-p_LT)*F_NL + tau_O)
# prog.AddLinearConstraint(I_M*dd_theta_M == tau_M-F_FM*(p_CN-p_MN))
# prog.AddLinearConstraint(F_NL == -F_NM)
# prog.AddLinearConstraint(F_FL == mu*mu_S*F_NL)
# prog.AddLinearConstraint(F_FM == -F_FL)
# prog.AddLinearConstraint(dd_d_s_T == dd_d_g_T)
# prog.AddLinearConstraint(dd_d_s_N == dd_d_g_N)

# Sympy 3

In [ ]:
import sympy as sp
import numpy as np
from IPython.display import display, Math, Latex
from sympy.utilities.lambdify import lambdify

In [ ]:
alg_inputs = []

# PROGRAMMING: Do this with Drake instead?
# PROGRAMMING: Can I get some of these lists from sympy?
# Physical and geometric quantities
m_L = sp.symbols(r"m_L")
alg_inputs.append(m_L)
m_M = sp.symbols(r"m_M")
alg_inputs.append(m_M)
w_L = sp.symbols(r"w_L")
alg_inputs.append(w_L)
I_L = sp.symbols(r"I_L")
alg_inputs.append(I_L)
h_L = sp.symbols(r"h_L")
alg_inputs.append(h_L)
r = sp.symbols(r"r")
alg_inputs.append(r)
I_M = sp.symbols(r"I_M")
alg_inputs.append(I_M)

In [ ]:
# Friction coefficients
mu = sp.symbols(r"\mu")
alg_inputs.append(mu)
mu_S = sp.symbols(r"\mu_{S}")
alg_inputs.append(mu_S)

# System gains
b_J = sp.symbols(r"b_J")
alg_inputs.append(b_J)
k_J = sp.symbols(r"k_J")
alg_inputs.append(k_J)

In [ ]:
# Positions
p_CN = sp.symbols(r"p_{CN}")
alg_inputs.append(p_CN)
p_CT = sp.symbols(r"p_{CT}")
alg_inputs.append(p_CT)
p_MN = sp.symbols(r"p_{MN}")
alg_inputs.append(p_MN)
p_LN = sp.symbols(r"p_{LN}")
alg_inputs.append(p_LN)
p_LT = sp.symbols(r"p_{LT}")
alg_inputs.append(p_LT)
theta_L = sp.symbols(r"\theta_L")
alg_inputs.append(theta_L)
d_T = sp.symbols(r"d_T")
alg_inputs.append(d_T)
d_N = sp.symbols(r"d_N")
alg_inputs.append(d_N)

# Velocities
v_MN = sp.symbols(r"v_{MN}")
alg_inputs.append(v_MN)
v_MT = sp.symbols(r"v_{MT}")
alg_inputs.append(v_MT)
v_LN = sp.symbols(r"v_{LN}")
alg_inputs.append(v_LN)
v_LT = sp.symbols(r"v_{LT}")
alg_inputs.append(v_LT)
d_theta_L = sp.symbols(r"\dot\theta_L")
alg_inputs.append(d_theta_L)
d_theta_M = sp.symbols(r"\dot\theta_M")
alg_inputs.append(d_theta_M)
d_d_T = sp.symbols(r"\dot{d}_T")
alg_inputs.append(d_d_T)
d_d_N = sp.symbols(r"\dot{d}_N")
alg_inputs.append(d_d_N)

In [ ]:
# Input forces
F_GT = sp.symbols(r"F_{GT}")
alg_inputs.append(F_GT)
F_GN = sp.symbols(r"F_{GN}")
alg_inputs.append(F_GN)
F_OT, F_ON, tau_O = sp.symbols(r"F_{OT}, F_{ON} \tau_O")
alg_inputs.append(F_OT)
alg_inputs.append(F_ON)
alg_inputs.append(tau_O)

# Control inputs
dd_theta_Ld = sp.symbols(r"\ddot\theta_{Ld}")
alg_inputs.append(dd_theta_Ld)
dd_d_Nd = sp.symbols(r"\ddot{d}_{Nd}")
alg_inputs.append(dd_d_Nd)
dd_d_Td = sp.symbols(r"\ddot{d}_{Td}")
alg_inputs.append(dd_d_Td)
dd_theta_Md = sp.symbols(r"\ddot\theta_{Md}")
alg_inputs.append(dd_theta_Md)

outputs = [
    a_LT, a_LN, a_MT, a_MN, dd_theta_L, dd_theta_M, F_NM, F_FL, F_FM, dd_d_N, dd_d_T, F_NL, F_CN, F_CT, tau_M
] = sp.symbols(
    r"a_{LT} a_{LN} a_{MT} a_{MN} \ddot\theta_L \ddot\theta_M  F_{NM} F_{FL} F_{FM} \ddot{d}_N \ddot{d}_T F_{NL} F_{CN}, F_{CT} \tau_M"
)

outputs = [
    a_LT, a_LN, a_MT, a_MN, dd_theta_L, dd_theta_M, F_NM, F_FL, F_FM, dd_d_N, dd_d_T, F_NL, F_CN, F_CT, tau_M
] = sp.symbols(
    r"a_{LT} a_{LN} a_{MT} a_{MN} \ddot\theta_L \ddot\theta_M  F_{NM} F_{FL} F_{FM} \ddot{d}_N \ddot{d}_T F_{NL} F_{CN}, F_{CT} \tau_M"
)

outputs = list(outputs)

In [ ]:
t = sp.symbols("t")
theta_L_func = sp.Function(r'\theta_L')(t)
N_hat = sp.Function(r'\hat N')(theta_L_func)
T_hat = sp.Function(r'\hat T')(theta_L_func)

d_T_func = sp.Function(r"d_T")(t)
d_N_func = sp.Function(r"d_N")(t)
d_g = d_T_func*T_hat + d_N_func*N_hat

d_vel_g = sp.diff(d_g, t)

d_vel_g = d_vel_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_g = d_vel_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_g = sp.diff(d_vel_g, t)
d_acc_g = d_acc_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_g = d_acc_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_g = d_acc_g
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_g = d_acc_cos_g.subs(d_T_func, d_T)
d_acc_cos_g = d_acc_cos_g.subs(d_N_func, d_N)

In [ ]:
dd_d_g_T = d_acc_cos_g.subs(N_hat, 0).subs(T_hat, 1)
dd_d_g_T

In [ ]:
dd_d_g_N = d_acc_cos_g.subs(T_hat, 0).subs(N_hat, 1)
dd_d_g_N

In [ ]:
p_M_func = sp.Function(r"p_M")(t)
p_L_func = sp.Function(r"p_L")(t)
v_M = sp.symbols(r"v_M")
v_L = sp.symbols(r"v_L")
d_s = (p_M_func + r*N_hat) - (p_L_func + (w_L/2)*T_hat - (h_L/2)*N_hat)

d_vel_s = sp.diff(d_s, t)
d_vel_s = d_vel_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_s = d_vel_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_s = sp.diff(d_vel_s, t)
d_acc_s = d_acc_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_s = d_acc_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_s = d_acc_s
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_s = d_acc_cos_s.subs(d_T_func, d_T)
d_acc_cos_s = d_acc_cos_s.subs(d_N_func, d_N)

In [ ]:
dd_d_s_T = d_acc_cos_s.subs(N_hat, 0).subs(T_hat, 1)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_L_func, t, t), a_LT)
dd_d_s_T

In [ ]:
dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)

In [ ]:
dd_d_s_N = d_acc_cos_s.subs(T_hat, 0).subs(N_hat, 1)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_M_func, t, t), a_MN)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_L_func, t, t), a_LN)
dd_d_s_N

In [ ]:
nat_eqs = [
    # Link tangential force balance
    [m_L*a_LT, F_FL+F_GT+F_OT],
    # Link normal force balance
    [m_L*a_LN, F_NL + F_GN + F_ON, ],
    # Manipulator tangential force balance
    [m_M*a_MT, F_FM + F_CT, ],
    # Manipulator normal force balance
    [m_M*a_MN, F_NM+F_CN, ],
    # Link moment balance
    [I_L*dd_theta_L, (-w_L/2)*F_ON - (p_CN-p_LN) * \
     F_FL + (p_CT-p_LT)*F_NL + tau_O, ],
    # Manipulator moment balance
    [I_M*dd_theta_M, tau_M-F_FM*(p_CN-p_MN), ],
    # 3rd law normal forces
    [F_NL, -F_NM],
    # Friction relationship L
    [F_FL, mu*mu_S*F_NL],
    # Friction relationship M
    [F_FM, -F_FL],
    # d_T derivative is derivative
    [dd_d_s_T, dd_d_g_T],
    # d_N derivative is derivative
    [dd_d_s_N, dd_d_g_N],
]

art_eqs = [
    [dd_d_N, dd_d_Nd],
    [dd_d_T, dd_d_Td],
    [dd_theta_L, dd_theta_Ld],
    [dd_theta_M, dd_theta_Md],
]

env_eqs = nat_eqs# + art_eqs

In [ ]:
out_str = r"\begin{aligned}"
for eq in env_eqs:
    out_str += sp.latex(eq[0]) + r" &= " + sp.latex(eq[1]) + r" \\" + "\n"
out_str += "\end{aligned}"
display(Math(out_str))

In [ ]:
sp.latex(nat_eqs[0][0])

In [ ]:
A = []
b = []
for lhs, rhs in env_eqs:
    A_row = []
    b_term = rhs - lhs
    for output_term in outputs:
        try:
            coeff_L = lhs.coeff(output_term)
        except AttributeError:
            coeff_L = 0
        try:
            coeff_R = rhs.coeff(output_term)
        except AttributeError:
            coeff_R = 0
        coeff = coeff_L - coeff_R
        A_row.append(coeff)
        b_term += coeff * output_term
    A.append(A_row)
    b.append(b_term)
A = sp.SparseMatrix(A)
A.simplify()
b = sp.Matrix([b]).T
b.simplify()
x = sp.Matrix([outputs]).T
x.simplify()

In [ ]:
A

In [ ]:
x

In [ ]:
# A_p = A.pinv()

In [ ]:
# A_aug = A.row_join(b)
# results = A_aug.rref()[0]
# A_prime = results[:, :-1]
# b_prime = results[:, -1]

# assert A_prime == sp.eye(A_prime.shape[0])

# F_CN_idx = list(x).index(F_CN)
# F_CN_exp = b_prime[F_CN_idx]
# F_CT_idx = list(x).index(F_CT)
# F_CT_exp = b_prime[F_CT_idx]
# tau_M_idx = list(x).index(tau_M)
# tau_M_exp = b_prime[tau_M_idx]

# get_F_CN = lambdify([alg_inputs], F_CN_exp)
# get_F_CT = lambdify([alg_inputs], F_CT_exp)
# get_tau_M = lambdify([alg_inputs], tau_M_exp)

# tau_M_idx = list(x).index(tau_M)
# tau_M_exp = b_prime[tau_M_idx]

In [ ]:
A@x

In [ ]:
x_old = x
k = b

In [ ]:
A_old = A

In [ ]:
A_old

In [ ]:
x_old

In [ ]:
x = sp.Matrix(x_old[:4] + x_old[6:9] + [x_old[11]])
A = A_old[:,:4].row_join(A_old[:,6:9]).row_join(A_old[:,11])

I_mu = sp.zeros(A.shape[0], A.shape[1])
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        I_mu[i,j] = mu*(A[i,j].coeff(mu))
A -= I_mu

In [ ]:
A

In [ ]:
u = sp.Matrix(x_old[-3:])
B = A_old[:,-3:]
B

In [ ]:
y_ddot = sp.Matrix(x_old[4:6]  + x_old[9:11])
C = A_old[:,4:6].row_join(A_old[:,9:11])
C

In [ ]:
CC_i = ((C.T)@C).inv()

In [ ]:
C_left_inverse = CC_i@(C.T)

In [ ]:
C.T@A

In [ ]:
C.T

In [ ]:
func = lambdify([list(C_left_inverse.free_symbols)], C_left_inverse)

In [ ]:
func([1, 2, 3, 4, 5, 5, 7])

In [ ]:
-A@x-B@u+k

In [ ]:
-A_old@x_old+b

In [ ]:
-A_old@x_old+b-(-A@x-B@u+k)

In [ ]:
vec = (C.T@(-A@x-B@u+k))
vec.simplify()
vec

In [ ]:
out_str = r"\begin{align}"
for eq in env_eqs:
    out_str += sp.latex(eq[0]) + r" &= " + sp.latex(eq[1]) + r" \\" + "\n"
out_str += "\end{align}"
display(Math(out_str))

In [ ]:
C.T@B

In [ ]:
v = sp.zeros(11, 1)
v[0] = 1
v[1] = 2
v[2] = 3
v[3] = 4
v

In [ ]:
C.T@(-A@x-A_mu@x -B@u+k)

In [ ]:
B

# Sympy 4

In [ ]:
import sympy as sp
import numpy as np
from IPython.display import display, Math, Latex
from sympy.utilities.lambdify import lambdify

In [ ]:
alg_inputs = []

# PROGRAMMING: Do this with Drake instead?
# PROGRAMMING: Can I get some of these lists from sympy?
# Physical and geometric quantities
m_L = sp.symbols(r"m_L")
alg_inputs.append(m_L)
m_M = sp.symbols(r"m_M")
alg_inputs.append(m_M)
w_L = sp.symbols(r"w_L")
alg_inputs.append(w_L)
I_L = sp.symbols(r"I_L")
alg_inputs.append(I_L)
h_L = sp.symbols(r"h_L")
alg_inputs.append(h_L)
r = sp.symbols(r"r")
alg_inputs.append(r)
I_M = sp.symbols(r"I_M")
alg_inputs.append(I_M)

In [ ]:
# Friction coefficients
mu = sp.symbols(r"\mu")
alg_inputs.append(mu)
mu_S = sp.symbols(r"\mu_{S}")
alg_inputs.append(mu_S)

# System gains
b_J = sp.symbols(r"b_J")
alg_inputs.append(b_J)
k_J = sp.symbols(r"k_J")
alg_inputs.append(k_J)

In [ ]:
# Positions
p_CN = sp.symbols(r"p_{CN}")
alg_inputs.append(p_CN)
p_CT = sp.symbols(r"p_{CT}")
alg_inputs.append(p_CT)
p_MN = sp.symbols(r"p_{MN}")
alg_inputs.append(p_MN)
p_LN = sp.symbols(r"p_{LN}")
alg_inputs.append(p_LN)
p_LT = sp.symbols(r"p_{LT}")
alg_inputs.append(p_LT)
theta_L = sp.symbols(r"\theta_L")
alg_inputs.append(theta_L)
d_T = sp.symbols(r"d_T")
alg_inputs.append(d_T)
d_N = sp.symbols(r"d_N")
alg_inputs.append(d_N)

# Velocities
v_MN = sp.symbols(r"v_{MN}")
alg_inputs.append(v_MN)
v_MT = sp.symbols(r"v_{MT}")
alg_inputs.append(v_MT)
v_LN = sp.symbols(r"v_{LN}")
alg_inputs.append(v_LN)
v_LT = sp.symbols(r"v_{LT}")
alg_inputs.append(v_LT)
d_theta_L = sp.symbols(r"\dot\theta_L")
alg_inputs.append(d_theta_L)
d_theta_M = sp.symbols(r"\dot\theta_M")
alg_inputs.append(d_theta_M)
d_d_T = sp.symbols(r"\dot{d}_T")
alg_inputs.append(d_d_T)
d_d_N = sp.symbols(r"\dot{d}_N")
alg_inputs.append(d_d_N)

In [ ]:
# Input forces
F_GT = sp.symbols(r"F_{GT}")
alg_inputs.append(F_GT)
F_GN = sp.symbols(r"F_{GN}")
alg_inputs.append(F_GN)
F_OT, F_ON, tau_O = sp.symbols(r"F_{OT}, F_{ON} \tau_O")
alg_inputs.append(F_OT)
alg_inputs.append(F_ON)
alg_inputs.append(tau_O)

# Control inputs
dd_theta_Ld = sp.symbols(r"\ddot\theta_{Ld}")
alg_inputs.append(dd_theta_Ld)
dd_d_Nd = sp.symbols(r"\ddot{d}_{Nd}")
alg_inputs.append(dd_d_Nd)
dd_d_Td = sp.symbols(r"\ddot{d}_{Td}")
alg_inputs.append(dd_d_Td)
dd_theta_Md = sp.symbols(r"\ddot\theta_{Md}")
alg_inputs.append(dd_theta_Md)

outputs = [
    a_LT, a_LN, a_MT, a_MN, dd_theta_L, dd_theta_M, F_NM, F_FL, F_FM, dd_d_N, dd_d_T, F_NL, F_CN, F_CT, tau_M
] = sp.symbols(
    r"a_{LT} a_{LN} a_{MT} a_{MN} \ddot\theta_L \ddot\theta_M  F_{NM} F_{FL} F_{FM} \ddot{d}_N \ddot{d}_T F_{NL} F_{CN}, F_{CT} \tau_M"
)

outputs = [
    a_LT, a_LN, a_MT, a_MN, dd_theta_L, dd_theta_M, F_NM, F_FL, F_FM, dd_d_N, dd_d_T, F_NL, F_CN, F_CT, tau_M
] = sp.symbols(
    r"a_{LT} a_{LN} a_{MT} a_{MN} \ddot\theta_L \ddot\theta_M  F_{NM} F_{FL} F_{FM} \ddot{d}_N \ddot{d}_T F_{NL} F_{CN}, F_{CT} \tau_M"
)

outputs = list(outputs)

In [ ]:
t = sp.symbols("t")
theta_L_func = sp.Function(r'\theta_L')(t)
N_hat = sp.Function(r'\hat N')(theta_L_func)
T_hat = sp.Function(r'\hat T')(theta_L_func)

d_T_func = sp.Function(r"d_T")(t)
d_N_func = sp.Function(r"d_N")(t)
d_g = d_T_func*T_hat + d_N_func*N_hat

d_vel_g = sp.diff(d_g, t)

d_vel_g = d_vel_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_g = d_vel_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_g = sp.diff(d_vel_g, t)
d_acc_g = d_acc_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_g = d_acc_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_g = d_acc_g
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_g = d_acc_cos_g.subs(d_T_func, d_T)
d_acc_cos_g = d_acc_cos_g.subs(d_N_func, d_N)

In [ ]:
dd_d_g_T = d_acc_cos_g.subs(N_hat, 0).subs(T_hat, 1)
dd_d_g_T

In [ ]:
dd_d_g_N = d_acc_cos_g.subs(T_hat, 0).subs(N_hat, 1)
dd_d_g_N

In [ ]:
p_M_func = sp.Function(r"p_M")(t)
p_L_func = sp.Function(r"p_L")(t)
v_M = sp.symbols(r"v_M")
v_L = sp.symbols(r"v_L")
d_s = (p_M_func + r*N_hat) - (p_L_func + (w_L/2)*T_hat - (h_L/2)*N_hat)

d_vel_s = sp.diff(d_s, t)
d_vel_s = d_vel_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_s = d_vel_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_s = sp.diff(d_vel_s, t)
d_acc_s = d_acc_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_s = d_acc_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_s = d_acc_s
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_s = d_acc_cos_s.subs(d_T_func, d_T)
d_acc_cos_s = d_acc_cos_s.subs(d_N_func, d_N)

In [ ]:
dd_d_s_T = d_acc_cos_s.subs(N_hat, 0).subs(T_hat, 1)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_L_func, t, t), a_LT)
dd_d_s_T

In [ ]:
dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)

In [ ]:
dd_d_s_N = d_acc_cos_s.subs(T_hat, 0).subs(N_hat, 1)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_M_func, t, t), a_MN)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_L_func, t, t), a_LN)
dd_d_s_N

In [ ]:
nat_eqs = [
    # Link tangential force balance
    [m_L*a_LT, F_FL+F_GT+F_OT],
    # Link normal force balance
    [m_L*a_LN, F_NL + F_GN + F_ON, ],
    # Manipulator tangential force balance
    [m_M*a_MT, F_FM + F_CT, ],
    # Manipulator normal force balance
    [m_M*a_MN, F_NM+F_CN, ],
    # Link moment balance
    [I_L*dd_theta_L, (-w_L/2)*F_ON - (p_CN-p_LN) * \
     F_FL + (p_CT-p_LT)*F_NL + tau_O, ],
    # Manipulator moment balance
    [I_M*dd_theta_M, tau_M-F_FM*(p_CN-p_MN), ],
    # 3rd law normal forces
    [F_NL, -F_NM],
    # Friction relationship L
    [F_FL, mu*mu_S*F_NL],
    # Friction relationship M
    [F_FM, -F_FL],
    # d_T derivative is derivative
    [dd_d_s_T, dd_d_g_T],
    # d_N derivative is derivative
    [dd_d_s_N, dd_d_g_N],
]

art_eqs = [
    [dd_d_N, dd_d_Nd],
    [dd_d_T, dd_d_Td],
    [dd_theta_L, dd_theta_Ld],
    [dd_theta_M, dd_theta_Md],
]

env_eqs = nat_eqs# + art_eqs

In [ ]:
out_str = r"\begin{aligned}"
for eq in env_eqs:
    out_str += sp.latex(eq[0]) + r" &= " + sp.latex(eq[1]) + r" \\" + "\n"
out_str += "\end{aligned}"
display(Math(out_str))

In [ ]:
sp.latex(nat_eqs[0][0])

In [ ]:
A = []
b = []
for lhs, rhs in env_eqs:
    A_row = []
    b_term = rhs - lhs
    for output_term in outputs:
        try:
            coeff_L = lhs.coeff(output_term)
        except AttributeError:
            coeff_L = 0
        try:
            coeff_R = rhs.coeff(output_term)
        except AttributeError:
            coeff_R = 0
        coeff = coeff_L - coeff_R
        A_row.append(coeff)
        b_term += coeff * output_term
    A.append(A_row)
    b.append(b_term)
A = sp.SparseMatrix(A)
A.simplify()
b = sp.Matrix([b]).T
b.simplify()
x = sp.Matrix([outputs]).T
x.simplify()

In [ ]:
A

In [ ]:
x

In [ ]:
# A_p = A.pinv()

In [ ]:
# A_aug = A.row_join(b)
# results = A_aug.rref()[0]
# A_prime = results[:, :-1]
# b_prime = results[:, -1]

# assert A_prime == sp.eye(A_prime.shape[0])

# F_CN_idx = list(x).index(F_CN)
# F_CN_exp = b_prime[F_CN_idx]
# F_CT_idx = list(x).index(F_CT)
# F_CT_exp = b_prime[F_CT_idx]
# tau_M_idx = list(x).index(tau_M)
# tau_M_exp = b_prime[tau_M_idx]

# get_F_CN = lambdify([alg_inputs], F_CN_exp)
# get_F_CT = lambdify([alg_inputs], F_CT_exp)
# get_tau_M = lambdify([alg_inputs], tau_M_exp)

# tau_M_idx = list(x).index(tau_M)
# tau_M_exp = b_prime[tau_M_idx]

In [ ]:
A@x

In [ ]:
x_old = x
k = b

In [ ]:
A_old = A

In [ ]:
A_old

In [ ]:
x_old

In [ ]:
x = sp.Matrix(x_old[:9] + [x_old[11]])
A = A_old[:,:9].row_join(A_old[:,11])

I_mu = sp.zeros(A.shape[0], A.shape[1])
for i in range(A.shape[0]):
    for j in range(A.shape[1]):
        I_mu[i,j] = mu*(A[i,j].coeff(mu))
A -= I_mu

In [ ]:
x

In [ ]:
A

In [ ]:
u = sp.Matrix(x_old[-3:])
B = A_old[:,-3:]
u

In [ ]:
B

In [ ]:
y_ddot = sp.Matrix(x_old[9:11])
C = A_old[:,9:11]
C

In [ ]:
C_np = np.zeros((11,2))
C_np[9, 1] = -1
C_np[10, 0] = -1
C_np

In [ ]:
C_np.T@C_np

In [ ]:
y_ddot

In [ ]:
C.T

In [ ]:
-A@x-B@u+k

In [ ]:
-A_old@x_old+b

In [ ]:
-A_old@x_old+b-(-A@x-B@u+k)

In [ ]:
vec = (C.T@(-A@x-B@u+k))
vec.simplify()
vec

In [ ]:
out_str = r"\begin{align}"
for eq in env_eqs:
    out_str += sp.latex(eq[0]) + r" &= " + sp.latex(eq[1]) + r" \\" + "\n"
out_str += "\end{align}"
display(Math(out_str))

In [ ]:
C.T@B

In [ ]:
v = sp.zeros(11, 1)
v[0] = 1
v[1] = 2
v[2] = 3
v[3] = 4
v

In [ ]:
C.T@(-A@x-A_mu@x -B@u+k)

In [ ]:
B

# Sympy 5

In [ ]:
import sympy as sp
import numpy as np
from IPython.display import display, Math, Latex
from sympy.utilities.lambdify import lambdify

In [ ]:
alg_inputs = []

# PROGRAMMING: Do this with Drake instead?
# PROGRAMMING: Can I get some of these lists from sympy?
# Physical and geometric quantities
m_L = sp.symbols(r"m_L")
alg_inputs.append(m_L)
m_M = sp.symbols(r"m_M")
alg_inputs.append(m_M)
w_L = sp.symbols(r"w_L")
alg_inputs.append(w_L)
I_L = sp.symbols(r"I_L")
alg_inputs.append(I_L)
h_L = sp.symbols(r"h_L")
alg_inputs.append(h_L)
r = sp.symbols(r"r")
alg_inputs.append(r)
I_M = sp.symbols(r"I_M")
alg_inputs.append(I_M)

In [ ]:
# Friction coefficients
mu = sp.symbols(r"\mu")
alg_inputs.append(mu)
mu_S = sp.symbols(r"\mu_{S}")
alg_inputs.append(mu_S)

# System gains
b_J = sp.symbols(r"b_J")
alg_inputs.append(b_J)
k_J = sp.symbols(r"k_J")
alg_inputs.append(k_J)

In [ ]:
# Positions
p_CN = sp.symbols(r"p_{CN}")
alg_inputs.append(p_CN)
p_CT = sp.symbols(r"p_{CT}")
alg_inputs.append(p_CT)
p_MN = sp.symbols(r"p_{MN}")
alg_inputs.append(p_MN)
p_LN = sp.symbols(r"p_{LN}")
alg_inputs.append(p_LN)
p_LT = sp.symbols(r"p_{LT}")
alg_inputs.append(p_LT)
theta_L = sp.symbols(r"\theta_L")
alg_inputs.append(theta_L)
d_T = sp.symbols(r"d_T")
alg_inputs.append(d_T)
d_N = sp.symbols(r"d_N")
alg_inputs.append(d_N)

p_LCN = sp.symbols(r"p_{LCN}")
p_LCT = sp.symbols(r"p_{LCT}")
p_MCN = sp.symbols(r"p_{MCN}")
p_MCT = sp.symbols(r"p_{MCT}")

# Velocities
v_MN = sp.symbols(r"v_{MN}")
alg_inputs.append(v_MN)
v_MT = sp.symbols(r"v_{MT}")
alg_inputs.append(v_MT)
v_LN = sp.symbols(r"v_{LN}")
alg_inputs.append(v_LN)
v_LT = sp.symbols(r"v_{LT}")
alg_inputs.append(v_LT)
d_theta_L = sp.symbols(r"\dot\theta_L")
alg_inputs.append(d_theta_L)
d_theta_M = sp.symbols(r"\dot\theta_M")
alg_inputs.append(d_theta_M)
d_d_T = sp.symbols(r"\dot{d}_T")
alg_inputs.append(d_d_T)
d_d_N = sp.symbols(r"\dot{d}_N")
alg_inputs.append(d_d_N)

In [ ]:
# Input forces
F_GT = sp.symbols(r"F_{GT}")
alg_inputs.append(F_GT)
F_GN = sp.symbols(r"F_{GN}")
alg_inputs.append(F_GN)
F_OT, F_ON, tau_O = sp.symbols(r"F_{OT}, F_{ON} \tau_O")
alg_inputs.append(F_OT)
alg_inputs.append(F_ON)
alg_inputs.append(tau_O)

# Control inputs
dd_theta_Ld = sp.symbols(r"\ddot\theta_{Ld}")
alg_inputs.append(dd_theta_Ld)
dd_d_Nd = sp.symbols(r"\ddot{d}_{Nd}")
alg_inputs.append(dd_d_Nd)
dd_d_Td = sp.symbols(r"\ddot{d}_{Td}")
alg_inputs.append(dd_d_Td)
dd_theta_Md = sp.symbols(r"\ddot\theta_{Md}")
alg_inputs.append(dd_theta_Md)

outputs = [
    a_LT, a_LN, a_MT, a_MN, F_NM, F_FL, F_FM, F_NL, F_CN, F_CT, tau_M, dd_theta_L, dd_theta_M, dd_d_N, dd_d_T
] = sp.symbols(
    r"a_{LT}, a_{LN}, a_{MT}, a_{MN}, F_{NM}, F_{FL}, F_{FM}, F_{NL}, F_{CN}, F_{CT}, \tau_M, \ddot\theta_L, \ddot\theta_M, \ddot{d}_N, \ddot{d}_T"
)

outputs = list(outputs)

In [ ]:
F_FL

In [ ]:
t = sp.symbols("t")
theta_L_func = sp.Function(r'\theta_L')(t)
N_hat = sp.Function(r'\hat N')(theta_L_func)
T_hat = sp.Function(r'\hat T')(theta_L_func)

d_T_func = sp.Function(r"d_T")(t)
d_N_func = sp.Function(r"d_N")(t)
d_g = d_T_func*T_hat + d_N_func*N_hat

d_vel_g = sp.diff(d_g, t)

d_vel_g = d_vel_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_g = d_vel_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_g = sp.diff(d_vel_g, t)
d_acc_g = d_acc_g.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_g = d_acc_g.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_g = d_acc_g
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_g = d_acc_cos_g.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_g = d_acc_cos_g.subs(d_T_func, d_T)
d_acc_cos_g = d_acc_cos_g.subs(d_N_func, d_N)

In [ ]:
dd_d_g_T = d_acc_cos_g.subs(N_hat, 0).subs(T_hat, 1)
dd_d_g_T

In [ ]:
dd_d_g_N = d_acc_cos_g.subs(T_hat, 0).subs(N_hat, 1)
dd_d_g_N

In [ ]:
p_M_func = sp.Function(r"p_M")(t)
p_L_func = sp.Function(r"p_L")(t)
v_M = sp.symbols(r"v_M")
v_L = sp.symbols(r"v_L")
d_s = (p_M_func + r*N_hat) - (p_L_func + (w_L/2)*T_hat - (h_L/2)*N_hat)

d_vel_s = sp.diff(d_s, t)
d_vel_s = d_vel_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_vel_s = d_vel_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_s = sp.diff(d_vel_s, t)
d_acc_s = d_acc_s.subs(sp.diff(N_hat, t), -
                       sp.diff(theta_L_func, t)*T_hat)
d_acc_s = d_acc_s.subs(
    sp.diff(T_hat, t), sp.diff(theta_L_func, t)*N_hat)

d_acc_cos_s = d_acc_s
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t, t), dd_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t, t), dd_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t, t), dd_d_N)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(theta_L_func, t), d_theta_L)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_T_func, t), d_d_T)
d_acc_cos_s = d_acc_cos_s.subs(sp.diff(d_N_func, t), d_d_N)
d_acc_cos_s = d_acc_cos_s.subs(d_T_func, d_T)
d_acc_cos_s = d_acc_cos_s.subs(d_N_func, d_N)

In [ ]:
dd_d_s_T = d_acc_cos_s.subs(N_hat, 0).subs(T_hat, 1)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)
dd_d_s_T = dd_d_s_T.subs(sp.diff(p_L_func, t, t), a_LT)
dd_d_s_T

In [ ]:
dd_d_s_T.subs(sp.diff(p_M_func, t, t), a_MT)

In [ ]:
dd_d_s_N = d_acc_cos_s.subs(T_hat, 0).subs(N_hat, 1)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_M_func, t, t), a_MN)
dd_d_s_N = dd_d_s_N.subs(sp.diff(p_L_func, t, t), a_LN)
dd_d_s_N

In [ ]:
nat_eqs = [
    # Link tangential force balance
    [m_L*a_LT, F_FL+F_GT+F_OT],
    # Link normal force balance
    [m_L*a_LN, F_NL + F_GN + F_ON, ],
    # Manipulator tangential force balance
    [m_M*a_MT, F_FM + F_CT, ],
    # Manipulator normal force balance
    [m_M*a_MN, F_NM+F_CN, ],
    # Link moment balance
    [I_L*dd_theta_L, (-w_L/2)*F_ON - (p_LCN) * \
     F_FL + (p_LCN)*F_NL + tau_O, ],
    # Manipulator moment balance
    [I_M*dd_theta_M, tau_M-F_FM*(p_MCN), ],
    # 3rd law normal forces
    [F_NL, -F_NM],
    # Friction relationship L
    [F_FL, mu*mu_S*F_NL],
    # Friction relationship M
    [F_FM, -F_FL],
    # d_T derivative is derivative
    [dd_d_s_T, dd_d_g_T],
    # d_N derivative is derivative
    [dd_d_s_N, dd_d_g_N],
]

art_eqs = [
#     [dd_d_N, dd_d_Nd],
#     [dd_d_T, dd_d_Td],
#     [dd_theta_L, dd_theta_Ld],
#     [dd_theta_M, dd_theta_Md],
]

env_eqs = nat_eqs + art_eqs

In [ ]:
out_str = r"\begin{aligned}"
for eq in env_eqs:
    out_str += sp.latex(eq[0]) + r" &= " + sp.latex(eq[1]) + r" \\" + "\n"
out_str += "\end{aligned}"
display(Math(out_str))

## manual algebra

In [ ]:
T_exp = dd_d_s_T - dd_d_g_T
T_exp.simplify()
T_exp_collected = sp.collect(T_exp, dd_theta_L)
T_exp_collected

## just kidding

In [ ]:
A = []
b = []
for lhs, rhs in env_eqs:
    A_row = []
    b_term = rhs - lhs
    for output_term in outputs:
        try:
            coeff_L = lhs.coeff(output_term)
        except AttributeError:
            coeff_L = 0
        try:
            coeff_R = rhs.coeff(output_term)
        except AttributeError:
            coeff_R = 0
        coeff = coeff_L - coeff_R
        A_row.append(coeff)
        b_term += coeff * output_term
    A.append(A_row)
    b.append(b_term)
A = sp.SparseMatrix(A)
A.simplify()
b = sp.Matrix([b]).T
b.simplify()
x = sp.Matrix([outputs]).T
x.simplify()

In [ ]:
A

In [ ]:
A_aug = A.row_join(b)
results = A_aug.rref()[0]
A_prime = results[:, :-1]
b_prime = results[:, -1]

In [ ]:
A_prime

In [ ]:
(A_prime@x)[-2:,:]

In [ ]:
A_prime_simpler = A_prime.copy()
fs = []
for i in range(A_prime_simpler.shape[0]):
    fs.append(sp.symbols(r"f_{" + str(i) + r"}"))
    A_prime_simpler[i, 11] = fs[i]

In [ ]:
A_prime_simpler

In [ ]:
x

In [ ]:
(A_prime_simpler@x)[7:,:]

### f[8]

In [ ]:
(A_prime_simpler@x)[8,:]

In [ ]:
f = A_prime[:, 11]
term = f[8]
term = sp.collect(term, p_LCN*m_L)
sp.collect(term, m_M).collect(d_T).collect(w_L)

In [ ]:
f[8].collect(p_LCN).collect(2*d_T).collect(w_L).collect(2*I_L).collect(m_L*m_M).collect(2*m_L)

In [ ]:
f8_term2 = f[8]*sp.fraction(f[8])[1]
f8_term2 = f8_term2.simplify()
f8_term2 -= 2*I_L*m_L+2*I_L*m_M
f8_term2 = f8_term2.simplify()
f8_term2 /=sp.fraction(f[8])[1]
f8_term2 = f8_term2.simplify()
f8_term2

In [ ]:
f_8_term1 = 2*I_L*(m_L+m_M)/sp.fraction(f[8])[1]
f_8_term1 = f_8_term1.collect(p_LCN).collect(2*m_L)
f_8_term1

### f[9]

In [ ]:
f[9].collect(p_LCN).collect(2*d_N).collect(h_L).collect(2*I_L).collect(m_L*m_M).collect(2*m_L).collect(2*r)

In [ ]:
f9_term2 = f[9]*sp.fraction(f[9])[1]
f9_term2.simplify()
f9_term2 -= 2*I_L*m_L*mu*mu_S+2*I_L*m_M*mu*mu_S
f9_term2.simplify()
f9_term2 /=sp.fraction(f[9])[1]
f9_term2.simplify()
f9_term2 = f9_term2.collect(p_LCN).collect(2*d_N).collect(h_L).collect(2*I_L).collect(m_L*m_M).collect(2*m_L).collect(2*r)
f9_term2 = f9_term2.simplify()
f9_term2

In [ ]:
f9_term2 = (f[9] - f9_term2).simplify()
f9_term2

In [ ]:
f9_term1 = 2*I_L*(m_L+m_M)/sp.fraction(f[8])[1]
f9_term1 = f9_term1.factor()
f9_term1

In [ ]:
num = sp.fraction(f[9])[0]
num1 = num - num.coeff(mu*mu_S)*(mu*mu_S - 1)
num1 = num1.simplify()
f9_term1 = num1/sp.fraction(f[9])[1]
f9_term1 = f9_term1.factor()
f9_term1

In [ ]:
f9_term2 = num.coeff(mu*mu_S).factor()/sp.fraction(f[9])[1].coeff(mu*mu_S)
f9_term2 = f9_term2.simplify()
f9_term2

In [ ]:
(f[9] - f9_term1 - f9_term2).simplify()

### b[8]

In [ ]:
b_prime[8].collect(mu*mu_S)

In [ ]:
num = sp.fraction(b_prime[8])[0]
num1 = num - num.coeff(mu*mu_S)*(mu*mu_S - 1)
num1 = num1.simplify()
num1.factor()

In [ ]:
b8_term1 = num1/sp.fraction(b_prime[8])[1]
b8_term1 = b8_term1.factor()
b8_term1

In [ ]:
b8_term2 = num.coeff(mu*mu_S).factor()/sp.fraction(b_prime[8])[1].coeff(mu*mu_S)
b8_term2 = b8_term2.simplify()
b8_term2

In [ ]:
(b_prime[8]-b8_term1-b8_term2).simplify()

## b[9]

In [ ]:
b_prime[9].collect(mu*mu_S)

In [ ]:
num = sp.fraction(b_prime[9])[0]
num1 = num - num.coeff(mu*mu_S)*(mu*mu_S - 1)
num1 = num1.simplify()
b9_term1 = num1/sp.fraction(b_prime[9])[1]
b9_term1 = b9_term1.factor()
b9_term1

In [ ]:
b9_term2 = num.coeff(mu*mu_S).factor()/sp.fraction(b_prime[9])[1].coeff(mu*mu_S)
b9_term2 = b9_term2.simplify()
b9_term2

In [ ]:
(b_prime[9]-b9_term1-b9_term2).simplify()

### other shit

In [ ]:
b_prime[7].collect(p_LCN)

In [ ]:
f[9].collect(p_LCN)

In [ ]:
sp.collect(f[9], mu)

In [ ]:

sp.collect(f[9], mu).subs(d_N, 0).subs(I_L, 1).subs(h_L, 11.1).subs(w_L, 1.1).subs(m_L, 2).subs(m_M, 3.6).subs(r, 55)

In [ ]:
f2 = sp.fraction(f[9])[1].coeff(mu)

In [ ]:
k = b_prime
k1 = sp.fraction(k[9])[0].coeff(mu)
k2 = sp.fraction(k[9])[1].coeff(mu)

In [ ]:
k1